## Transit Bunching 
* `cd data-analyses/rt_segment_speeds && pip install -r requirements.txt && cd ../_shared_utils && make setup_env && cd ../gtfs_digest`
* [Issue](https://github.com/cal-itp/data-analyses/issues/1099)
### 11/8
* Figure out how to address City of Visalia: one of the buses that is scheduled to arrive earlier arrives later than another bus. 
* This leads to a negative time stamp and makes it appear like there is a lot of bunching per the Transit Matters approach.



In [17]:
import datetime as dt

import altair as alt
import geopandas as gpd
import merge_data
import numpy as np
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling, helpers, time_series_utils
from shared_utils import catalog_utils, rt_dates, rt_utils
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS

# https://github.com/cal-itp/data-analyses/blob/main/_shared_utils/shared_utils/gtfs_analytics_data.yml
GTFS_DATA_DICT = catalog_utils.get_catalog("gtfs_analytics_data")

from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    GTFS_DATA_DICT,
    PROJECT_CRS,
    RT_SCHED_GCS,
    SCHED_GCS,
    SEGMENT_GCS,
)

In [18]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [19]:
may_date = "2024-05-22"

In [20]:
drop_for_preview = [
    "schedule_gtfs_dataset_key",
    "trip_instance_key",
    "shape_array_key",
    "feed_key",
    "trip_id",
]

### Grab Sample Routes

In [21]:
subset = [
    "schedule_gtfs_dataset_key",
    "route_id",
    "direction_id",
    "route_primary_direction",
    "service_date",
    "frequency",
]

In [22]:
GTFS_DATA_DICT.rt_vs_schedule_tables.sched_route_direction_metrics

'schedule_route_dir/schedule_route_direction_metrics'

In [23]:
route_dir_columns = [
    "schedule_gtfs_dataset_key",
    "route_id",
    "direction_id",
    "time_period",
    "route_primary_direction",
    "frequency",
    "service_date",
]

In [24]:
route_dir = merge_data.concatenate_schedule_by_route_direction([may_date])[
    route_dir_columns
]

In [25]:
route_dir.head()

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,route_primary_direction,frequency,service_date
0,015d67d5b75b5cf2b710bbadadfb75f5,17,0.00,all_day,Northbound,0.92,2024-05-22
1,015d67d5b75b5cf2b710bbadadfb75f5,17,0.00,offpeak,Northbound,0.62,2024-05-22
2,015d67d5b75b5cf2b710bbadadfb75f5,17,0.00,peak,Northbound,1.50,2024-05-22
3,015d67d5b75b5cf2b710bbadadfb75f5,17,1.00,all_day,Southbound,0.92,2024-05-22
4,015d67d5b75b5cf2b710bbadadfb75f5,17,1.00,offpeak,Southbound,0.69,2024-05-22


#### Attach operators and districts

In [26]:
# Grab Crosswalk
CROSSWALK = GTFS_DATA_DICT.schedule_tables.gtfs_key_crosswalk

In [27]:
crosswalk_cols = [
    "schedule_gtfs_dataset_key",
    "organization_name",
    "name",
    "caltrans_district",
]

In [28]:
crosswalk_df = (
    time_series_utils.concatenate_datasets_across_dates(
        SCHED_GCS, CROSSWALK, [may_date], data_type="df", columns=crosswalk_cols
    )
    .sort_values(["service_date"])
    .reset_index(drop=True)
)

In [29]:
crosswalk_df.head(2)

,schedule_gtfs_dataset_key,organization_name,name,caltrans_district,service_date
0,1770249a5a2e770ca90628434d4934b1,Ventura County Transportation Commission,VCTC GMV Schedule,07 - Los Angeles,2024-05-22
1,f8102a9c0693206bf36d302540bf1bcf,City of Corona,Corona Schedule,08 - San Bernardino,2024-05-22


In [30]:
crosswalk_df.shape

(189, 5)

In [31]:
routes = pd.merge(
    route_dir,
    crosswalk_df,
    on=["schedule_gtfs_dataset_key", "service_date"],
    how="left",
)

In [32]:
# routes = pd.concat([thousand_oaks, visalia, metro, metro_33])

In [33]:
routes.route_id.unique()

array(['17', '219', '228', ..., '10867636', '10867637', '11096761'],
      dtype=object)

In [34]:
routes.head(3)

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,route_primary_direction,frequency,service_date,organization_name,name,caltrans_district
0,015d67d5b75b5cf2b710bbadadfb75f5,17,0.00,all_day,Northbound,0.92,2024-05-22,Marin County Transit District,Bay Area 511 Marin Schedule,04 - Oakland
1,015d67d5b75b5cf2b710bbadadfb75f5,17,0.00,offpeak,Northbound,0.62,2024-05-22,Marin County Transit District,Bay Area 511 Marin Schedule,04 - Oakland
2,015d67d5b75b5cf2b710bbadadfb75f5,17,0.00,peak,Northbound,1.50,2024-05-22,Marin County Transit District,Bay Area 511 Marin Schedule,04 - Oakland


### Add Trips

In [35]:
TABLE = GTFS_DATA_DICT.schedule_downloads.trips

In [36]:
FILE = f"{COMPILED_CACHED_VIEWS}{TABLE}_{may_date}.parquet"

In [37]:
trips_subset = [
    "gtfs_dataset_key",
    "route_id",
    "trip_instance_key",
    "shape_array_key",
    "feed_key",
    "route_long_name",
    "direction_id",
    "route_type",
]

In [38]:
trips = pd.read_parquet(FILE)[trips_subset].rename(
    columns={"gtfs_dataset_key": "schedule_gtfs_dataset_key"}
)

In [39]:
trips_routes = pd.merge(
    trips,
    routes,
    on=["schedule_gtfs_dataset_key", "route_id", "direction_id"],
    how="inner",
)

In [40]:
trips_routes.shape

(392497, 15)

In [41]:
trips_routes.route_id.nunique()

1338

#### I know we can get this from the warehouse but it seems cumbersome. Correct me if I'm wrong.

In [42]:
# https://gtfs.org/documentation/schedule/reference/#
route_type_crosswalk = {
    "route_type": ["0", "1", "2", "3", "4", "5", "6", "7", "11", "12"],
    "route_type_str": [
        "Tram, Streetcar, Light rail",
        "Subway, Metro",
        "Rail",
        "Bus",
        "Ferry.",
        "Cable tram.",
        "Aerial lift, suspended cable car (e.g., gondola lift, aerial tramway).",
        "Funicular.",
        "Trolleybus.",
        "Monorail.",
    ],
}

In [43]:
route_type_crosswalk_df = pd.DataFrame(route_type_crosswalk)

In [44]:
# Merge for route_type
trips_routes = pd.merge(
    trips_routes, route_type_crosswalk_df, on=["route_type"], how="left"
)

In [45]:
trips_routes = trips_routes.drop(columns=["route_type"])

In [46]:
trips_routes = trips_routes.rename(columns={"route_type_str": "route_type"})

### Get Stop Times 

In [47]:
rt_stop_times = pd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/rt_vs_schedule/schedule_rt_stop_times_2024-05-22.parquet"
)

In [48]:
rt_stop_times.head(2)

,trip_id,stop_id,stop_sequence,scheduled_arrival_sec,schedule_gtfs_dataset_key,trip_instance_key,rt_arrival_sec
0,1d105244-776c-4b3f-af78-9c7ad78c2103,0b2443b6-b50f-452b-a749-464588ca93b8,8,60991.00,1fd2f07342d966919b15d5d37fda8cc8,45ae17540ca9fb5030c84dbb12e48e9a,61434
1,1d105244-776c-4b3f-af78-9c7ad78c2103,cd5650b0-9a18-4e78-aedc-385f3094fa0f,9,61179.00,1fd2f07342d966919b15d5d37fda8cc8,45ae17540ca9fb5030c84dbb12e48e9a,61616


In [49]:
trips_routes_times = pd.merge(
    rt_stop_times,
    trips_routes,
    on=[
        "schedule_gtfs_dataset_key",
        "trip_instance_key",
    ],
    how="inner",
)

In [50]:
(trips_routes_times.scheduled_arrival_sec.isna().sum())

44988

In [51]:
trips_routes_times.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11031910 entries, 0 to 11031909
Data columns (total 20 columns):
 #   Column                     Dtype         
---  ------                     -----         
 0   trip_id                    object        
 1   stop_id                    object        
 2   stop_sequence              int64         
 3   scheduled_arrival_sec      float64       
 4   schedule_gtfs_dataset_key  object        
 5   trip_instance_key          object        
 6   rt_arrival_sec             int64         
 7   route_id                   object        
 8   shape_array_key            object        
 9   feed_key                   object        
 10  route_long_name            object        
 11  direction_id               float64       
 12  time_period                object        
 13  route_primary_direction    object        
 14  frequency                  float64       
 15  service_date               datetime64[ns]
 16  organization_name          object 

### Sorting 

In [52]:
trips_routes_times2 = trips_routes_times.sort_values(
    by=[
        "schedule_gtfs_dataset_key",
        "route_long_name",
        "shape_array_key",
        "direction_id",
        "stop_sequence",
        "stop_id",
        "rt_arrival_sec",
    ]
).reset_index(drop=True)

In [53]:
preview_sort_col = [
    "schedule_gtfs_dataset_key",
    "route_id",
    "direction_id",
    "stop_sequence",
    "rt_arrival_sec",
    "stop_id",
    "scheduled_arrival_sec",
]

In [54]:
trips_routes_times2.loc[
    (
        trips_routes_times2.schedule_gtfs_dataset_key
        == "0666caf3ec1ecc96b74f4477ee4bc939"
    )
    & (trips_routes_times2.route_id == "204-13172")
    & (trips_routes_times2.direction_id == 1)
][preview_sort_col].shape

(21009, 7)

### Convert scheduled and RT arrival times.
* If 82800  < `scheduled_arrival_time` < 86_400 but `rt_arrival_sec` is lower say 14_000 (4 am in the morning): then the bus was scheduled to arrive on May 21 (day before the service date) but it arrived a little later on the actual service date we query the data for. 
* If  86_400 < `scheduled_arrival_time` and `rt_arrival_sec` is around 86_000 then this is the same service date. 

In [55]:
trips_routes_times2["rt_arrival_sec"].describe()

count   11031910.00
mean       48176.39
std        17806.20
min            0.00
25%        33526.00
50%        48304.00
75%        62400.00
max        86399.00
Name: rt_arrival_sec, dtype: float64

In [56]:
trips_routes_times2.loc[trips_routes_times2["scheduled_arrival_sec"] == 86_400].shape

(1024, 20)

####  Filter out for trips with `scheduled_arrival_sec` that's over 24 hours.

In [60]:
timestamp_subset = [
    "converted_schd_arrival",
    "converted_rt_arrival",
    "scheduled_arrival_sec",
    "rt_arrival_sec",
    "service_date",
]

In [61]:
def adjust_sched_arrival_seconds(sched_arrival_seconds, date, rt_arrival_sec):
    """
    Adjusts days and time of sched_arrival_seconds because it runs over 24 hours
    based on a combination of sched_arrival_seconds and rt_arrival_sec criteria.

    Parameters:
    sched_arrival_seconds (int): Number of seconds.
    date (datetime): Initial date.
    rt_arrival_sec (int): Arrival time in seconds.

    Returns:
    datetime: Adjusted date and time for sched_arrival_seconds
    """
    # If the rt arrival second is between 12-1AM and the scheduled arrival time is between
    # is between 11pm and 1am subtract a day
    if rt_arrival_sec < (60 * 60) and (82_800 < sched_arrival_seconds < 90_000):
        return pd.Timestamp(date + pd.Timedelta(days=-1)) + pd.Timedelta(
            seconds=sched_arrival_seconds % 86400
        )
    else:
        # No change
        return pd.Timestamp(date) + pd.Timedelta(seconds=sched_arrival_seconds)

#### Subset to make the df smaller


In [62]:
subset = [
    "stop_id",
    "stop_sequence",
    "scheduled_arrival_sec",
    "schedule_gtfs_dataset_key",
    "trip_instance_key",
    "rt_arrival_sec",
    "route_id",
    "shape_array_key",
    "route_long_name",
    "direction_id",
    "organization_name",
    "caltrans_district",
    "service_date",
    "route_type",
    "feed_key",
]

In [63]:
trips_routes_times2 = trips_routes_times2[subset]

In [65]:
#trips_routes_times2["converted_schd_arrival2"] = trips_routes_times2.apply(
 #   lambda row: adjust_sched_arrival_seconds(
   #     row["scheduled_arrival_sec"], row["service_date"], row["rt_arrival_sec"]
 #   ),
#    axis=1,
#)#

In [66]:
trips_routes_times2["converted_rt_arrival"] = pd.to_datetime(
    trips_routes_times2["service_date"]
) + pd.to_timedelta(trips_routes_times2["rt_arrival_sec"] % 86400, unit="s")

In [67]:
trips_routes_times2["converted_schd_arrival"] = pd.to_datetime(
    trips_routes_times2["service_date"]
) + pd.to_timedelta(trips_routes_times2["scheduled_arrival_sec"] % 86400, unit="s")

#### Checkout results from the code
* Make sure it makes sense.


In [69]:
trips_routes_times2.loc[trips_routes_times2["scheduled_arrival_sec"] > 86_400][
    [
        "scheduled_arrival_sec",
        "converted_schd_arrival",
        "rt_arrival_sec",
        "converted_rt_arrival",
    ]
].sample(20)

,scheduled_arrival_sec,converted_schd_arrival,rt_arrival_sec,converted_rt_arrival
1575402,87180.00,2024-05-22 00:13:00,846,2024-05-22 00:14:06
573074,91800.00,2024-05-22 01:30:00,5396,2024-05-22 01:29:56
4038881,103973.00,2024-05-22 04:52:53,17625,2024-05-22 04:53:45
1823671,102300.00,2024-05-22 04:25:00,15783,2024-05-22 04:23:03
4312349,86691.00,2024-05-22 00:04:51,209,2024-05-22 00:03:29
571749,89160.00,2024-05-22 00:46:00,2689,2024-05-22 00:44:49
4083191,98040.00,2024-05-22 03:14:00,11699,2024-05-22 03:14:59
4046767,87111.00,2024-05-22 00:11:51,595,2024-05-22 00:09:55
7397112,86820.00,2024-05-22 00:07:00,397,2024-05-22 00:06:37
772826,89160.00,2024-05-22 00:46:00,4180,2024-05-22 01:09:40


### Deal with delays
* Some very extreme values.

In [ ]:
more_than_86400 = deal_with_23_hours(trips_routes_times2)

count   10986922.00
mean           1.90
std           35.79
min        -1439.78
25%           -0.23
50%            1.45
75%            4.05
max         1439.98
Name: delay_min, dtype: float64

In [ ]:
percentiles = [0.01, 0.02, 0.05, 0.1, 0.9, 0.95, 0.98, 0.99]

### Help, can't fix everythin...how to address these edge cases? 

In [ ]:
trips_routes_times2.converted_schd_arrival.describe()

In [ ]:
trips_routes_times2[
    trips_routes_times2["converted_schd_arrival"].dt.strftime("%Y-%m-%d")
    == "2024-05-21"
].head(2)

In [ ]:
trips_routes_times2[
    trips_routes_times2["converted_schd_arrival"].dt.strftime("%Y-%m-%d")
    == "2024-05-23"
].head(2)

In [ ]:
trips_routes_times2.converted_rt_arrival.describe()

#### Question: Last time, I received the suggestion to throw away things more than 2 hrs because that is not bunching. But wouldn't this be misleading as a metric when we want to calculate all the rows?

In [ ]:
print(trips_routes_times2.delay_min.describe(percentiles))

In [ ]:
# trips_routes_times2 = trips_routes_times2.loc[
#    trips_routes_times2.delay_min < 120
# ].reset_index(drop=True)

In [ ]:
# trips_routes_times2 = trips_routes_times2.loc[
#    trips_routes_times2.delay_min > -120
# ].reset_index(drop=True)

### Calculate the actual & scheduled headway the `operator-route-direction_id-stop_sequence-stop_id-` grain


In [ ]:
groupby_cols = [
    "schedule_gtfs_dataset_key",
    "route_long_name",
    "direction_id",
    "stop_id",
    "stop_sequence",
]

In [ ]:
trips_routes_times2["actual_arrival_lag_min"] = (
    trips_routes_times2.groupby(groupby_cols)["converted_rt_arrival"]
    .diff()
    .dt.total_seconds()
    / 60
)

In [ ]:
trips_routes_times2["scheduled_arrival_lag_min"] = (
    trips_routes_times2.groupby(groupby_cols)["converted_schd_arrival"]
    .diff()
    .dt.total_seconds()
    / 60
)

In [ ]:
trips_routes_times2["scheduled_arrival_lag_min"].describe(percentiles)

In [ ]:
trips_routes_times2["actual_arrival_lag_min"].describe(percentiles)

In [ ]:
preview_time_col = [
    "schedule_gtfs_dataset_key",
    "route_id",
    "stop_id",
    "stop_sequence",
    "converted_rt_arrival",
    "actual_arrival_lag_min",
    "converted_schd_arrival",
    "scheduled_arrival_lag_min",
]

In [ ]:
trips_routes_times2.loc[
    (
        trips_routes_times2.schedule_gtfs_dataset_key
        == "0666caf3ec1ecc96b74f4477ee4bc939"
    )
    & (trips_routes_times2.route_id == "204-13172")
    & (trips_routes_times2.stop_id == "3961")
][preview_time_col]

### Many lags are actually empty b/c it's the first of that groupby-sequence.

In [ ]:
trips_routes_times2[trips_routes_times2["scheduled_arrival_lag_min"].isna()][
    preview_time_col
].sample()

In [ ]:
trips_routes_times2.loc[
    (
        trips_routes_times2.schedule_gtfs_dataset_key
        == "0666caf3ec1ecc96b74f4477ee4bc939"
    )
    & (trips_routes_times2.route_id == "204-13172")
    & (trips_routes_times2.stop_sequence == 2)
][preview_time_col]

In [ ]:
trips_routes_times2.loc[
    (
        trips_routes_times2.schedule_gtfs_dataset_key
        == "0666caf3ec1ecc96b74f4477ee4bc939"
    )
    & (trips_routes_times2.route_id == "204-13172")
    & (trips_routes_times2.stop_sequence == 59)
][preview_time_col]

In [ ]:
trips_routes_times2.loc[
    (
        trips_routes_times2.schedule_gtfs_dataset_key
        == "0666caf3ec1ecc96b74f4477ee4bc939"
    )
    & (trips_routes_times2.route_id == "204-13172")
    & (trips_routes_times2.stop_sequence == 46)
    & (trips_routes_times2.stop_id == "5685")
][preview_time_col]

In [ ]:
# rt_stop_times4 = rt_stop_times4.fillna(0)

### Transit Matters Method

In [ ]:
transit_matters_df1 = trips_routes_times2.copy()

In [ ]:
transit_matters_df1["pct_actual_schd_headway"] = (
    transit_matters_df1.actual_arrival_lag_min
    / transit_matters_df1.scheduled_arrival_lag_min
)

In [ ]:
transit_matters_df1["bunched_y_n"] = np.where(
    transit_matters_df1["pct_actual_schd_headway"] < 0.25, "bunched", "not bunched"
)

In [ ]:
transit_matters_df1.bunched_y_n.value_counts() / len(transit_matters_df1)

#### Aggregate.
* At this point, it doesn't matter the sequence, we just care about how bunched the traffic is around one partiuclar stop. 
* See how many trips for that grain are considered "bunched" or not.

In [ ]:
def bunched_not_bunched(
    df: pd.DataFrame, bunched_y_n: str, groupby_cols: list
) -> pd.DataFrame:
    df2 = df.loc[df.bunched_y_n == bunched_y_n].reset_index(drop=True)

    bunched_y_n = bunched_y_n.replace(" ", "_")
    agg1 = (
        df2.groupby(groupby_cols).agg({"trip_instance_key": "nunique"}).reset_index()
    ).rename(columns={"trip_instance_key": f"{bunched_y_n}_trips"})
    return agg1

In [ ]:
def agg_final_df(df: pd.DataFrame) -> pd.DataFrame:
    groupby_cols = [
        "caltrans_district",
        "schedule_gtfs_dataset_key",
        "feed_key",
        "organization_name",
        "route_long_name",
        "route_type",
        "route_id",
        "direction_id",
        "stop_id",
        "stop_sequence",
    ]

    # Find total trips that are bunched
    bunched = bunched_not_bunched(df, "bunched", groupby_cols)

    # Find total trips that are NOT bunched
    not_bunched = bunched_not_bunched(df, "not bunched", groupby_cols)

    # Merge
    m1 = pd.merge(not_bunched, bunched, on=groupby_cols, how="outer")

    # Find the % of bunched trips
    m1 = m1.fillna(0)
    m1["all_trips"] = m1.not_bunched_trips + m1.bunched_trips
    m1["per_trip_bunched_per_stop"] = m1.bunched_trips / m1.all_trips

    # Filter out any rows with only one trip of that groupby combo
    # for that service date
    m1 = m1.loc[m1.all_trips > 1].reset_index(drop=True)
    m1 = m1.drop(columns=["not_bunched_trips", "bunched_trips"])

    return m1

In [ ]:
transit_matters_m1 = agg_final_df(transit_matters_df1)

In [ ]:
# transit_matters_m1 = (
#   transit_matters_m1.sort_values(by=["all_trips"], ascending=False)
#   .drop_duplicates(subset=transit_matters_agg)
#   .reset_index(drop=True)
# )

### Help: Swapped order of a bus is messing with the transit matters metric.
* How to solve for this?? 

In [ ]:
preview_cols = [
    "converted_rt_arrival",
    "actual_arrival_lag_min",
    "converted_schd_arrival",
    "scheduled_arrival_lag_min",
    "pct_actual_schd_headway",
    "bunched_y_n",
]

In [ ]:
example2 = transit_matters_df1.loc[
    (transit_matters_df1.stop_id == "2307719")
    & (transit_matters_df1.organization_name == "City of Visalia")
    & (transit_matters_df1.route_id == "2042")
    & (transit_matters_df1.shape_array_key == "60da59c7000ea5dcb5f845d8fa227f14")
]

#### Starting row 33484: the RT Arrival time is swapped. A bus that was scheduled to arrive at 4:27 arrived boefre the bus arrived at 3:42.
* This repeats again row 33486.

In [ ]:
example2[preview_cols]

### Use 2 minute benchmark
* [Source](https://static1.squarespace.com/static/533b9a24e4b01d79d0ae4376/t/645e82de1f570b31497c44dc/1683915486889/TransitMatters-Headwaymanagement.pdf)
* Justifying the use of
headway maintenance. For example, in April
2022 the 66 bus significantly bunched around
several stops. When bunching is defined as
buses that run within two minutes or less of
each other, inbound buses towards Nubian
Square bunched 10% of the time at Brigham
Circle, 9% at Brookline Village and Roxbury
Crossing, and 8% of the time at Coolidge
Corner. Bunching is even more dramatic
outbound towards Harvard Square where
buses bunched over 35% of the time at Winship
St, 13% at Coolidge Corner and Harvard Ave at
Commonwealth Ave, and 12% at North Harvard
St at Western Ave. View more data about bus
bunching through the TransitMatters Data
Dashboard here.

* To Do: add back in route  & operator information

In [ ]:
two_minutes_df = trips_routes_times2.copy()

In [ ]:
two_minutes_df["bunched_y_n"] = np.where(
    two_minutes_df["actual_arrival_lag_min"] <= 2, "bunched", "not bunched"
)

In [ ]:
two_minutes_df.bunched_y_n.value_counts()

In [ ]:
final_two_minute = agg_final_df(two_minutes_df)

In [ ]:
final_two_minute.loc[
    (final_two_minute.stop_id == "2307695")
    & (final_two_minute.organization_name == "City of Visalia")
    & (final_two_minute.route_id == "2042")
]

### Comparing both outcomes
* There are so many more bunched trips for the 2 minute approach.
* Add back in schedule_gtfs_key and then grab stop level data from the warehouse.

In [ ]:
final_two_minute.per_trip_bunched_per_stop.describe(percentiles)

In [ ]:
transit_matters_m1.per_trip_bunched_per_stop.describe(percentiles)

### Make Visuals

In [ ]:
freq_range = [
    "#ccbb44",
    "#e9d868",
    "#fcb40e",
    "#ff9c42",
    "#fc5c04",
    "#dd217d",
    "#dd217d",
    "#dd217d",
]

In [ ]:
trips_routes_times2["hour"] = trips_routes_times2["converted_rt_arrival"].dt.hour
trips_routes_times2["min"] = trips_routes_times2["converted_rt_arrival"].dt.minute

In [ ]:
trips_routes_times2.head(2)

In [ ]:
def compare_approaches(
    stop_id: str, organization_name: str, route_id: str, stop_sequence: int
):
    transit_matter = transit_matters_m1.loc[
        (transit_matters_m1.stop_id == stop_id)
        & (transit_matters_m1.organization_name == organization_name)
        & (transit_matters_m1.route_id == route_id)
        & (transit_matters_m1.stop_sequence == stop_sequence)
    ]
    display(transit_matter)

    two_min = final_two_minute.loc[
        (final_two_minute.stop_id == stop_id)
        & (final_two_minute.organization_name == organization_name)
        & (final_two_minute.route_id == route_id)
        & (final_two_minute.stop_sequence == stop_sequence)
    ]

    display(two_min)
    total_trips = trips_routes_times2.loc[
        (trips_routes_times2.stop_id == stop_id)
        & (trips_routes_times2.organization_name == organization_name)
        & (trips_routes_times2.route_id == route_id)
        & (trips_routes_times2.stop_sequence == stop_sequence)
    ]

    display(total_trips.trip_instance_key.nunique())

    chart = (
        alt.Chart(total_trips)
        .mark_circle(size=500)
        .encode(
            x="hour",
            y="min",
            color=alt.Color(
                "hour",
                scale=alt.Scale(range=freq_range),
            ),
            tooltip=["hour", "min", "actual_arrival_lag_min"],
        )
        .properties(width=800, height=400)
    )
    display(chart)
    return total_trips

In [ ]:
test1 = compare_approaches(
    stop_id="5685",
    organization_name="Los Angeles County Metropolitan Transportation Authority",
    route_id="204-13172",
    stop_sequence=46,
)

In [ ]:
test2 = compare_approaches(
    stop_id="2307469",
    organization_name="City of Visalia",
    route_id="2042",
    stop_sequence=27,
)

In [ ]:
test3 = compare_approaches(
    stop_id="3104",
    organization_name="Los Angeles County Metropolitan Transportation Authority",
    route_id="33-13172",
    stop_sequence=80,
)

In [ ]:
test4 = compare_approaches(
    stop_id="15320",
    organization_name="Los Angeles County Metropolitan Transportation Authority",
    route_id="33-13172",
    stop_sequence=64,
)

In [ ]:
test5 = compare_approaches(
    stop_id="3288014",
    organization_name="City of Thousand Oaks",
    route_id="3402",
    stop_sequence=16,
)

In [ ]:
transit_matters_m1.sort_values(by=["per_trip_bunched_per_stop"], ascending=False)

### Make Maps 
* Think I actually need the vehicle positions since stops are literally the stop's geometry, so it'll always be plotting on the same spot.

* https://github.com/cal-itp/data-analyses/blob/db19b70329f1e817236bda13707dd903c24abb4c/_shared_utils/shared_utils/gtfs_utils_v2.py#L371
* https://github.com/cal-itp/data-analyses/blob/main/gtfs_funnel/download_stops.py

In [ ]:
stop

In [ ]:
# What is this file?
vps_gdf = gpd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/rt_segment_speeds/vp_2024-05-22.parquet"
)

In [ ]:
vps_gdf = vps_gdf[
    [
        "schedule_gtfs_dataset_key",
        "trip_instance_key",
        "location_timestamp_local",
        "geometry",
    ]
]

In [ ]:
vps_df = vps_gdf[
    [
        "schedule_gtfs_dataset_key",
        "trip_instance_key",
        "location_timestamp_local",
    ]
]

In [ ]:
vps_m1 = pd.merge(
    vps_df,
    trips_routes_times2,
    on=["schedule_gtfs_dataset_key", "trip_instance_key"],
    how="inner",
)

In [ ]:
def one_stop(df: pd.DataFrame, stop_id: str, org_name: str, route_id: str):
    # Look at one route & stop
    test_route1 = df.loc[
        (df.organization_name == org_name)
        & (df.route_id == route_id)
        & (df.stop_id == stop_id)
    ]

    compare_approaches(stop_id=stop_id, organization_name=org_name, route_id=route_id)

    # display(test_route1.explore("time_int", marker_kwds = {'radius':20}))
    return test_route1

In [ ]:
metro_test1 = one_stop(
    vps_m1,
    stop_id="5700",
    org_name="Los Angeles County Metropolitan Transportation Authority",
    route_id="204-13172",
)

In [ ]:
metro_test1.shape

In [ ]:
metro_test1.sample(1)

### Other
* https://www.sciencedirect.com/science/article/pii/S1366554523003666
* https://www.sciencedirect.com/science/article/pii/S0968090X22002492?ref=pdf_download&fr=RR-2&rr=8d7d6fb73d8015be